In [1]:
import numpy as np
# Load the "autoreload" extension so that code can change
%load_ext autoreload

#always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.data import make_dataset

In [2]:
path = "/Users/jbaripatti/machine_learning_proj/data/interim/"
name = 'data_interim.txt'
filepath = path + name
data = np.loadtxt(filepath, delimiter=",")
X_train, y_train, X_test, y_test = make_dataset.split_data(data)

In [3]:
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.models import Sequential

Using TensorFlow backend.


In [5]:
#normalize training set, then will use the same transformation to the test set.
from sklearn.preprocessing import Normalizer
X_train = Normalizer().fit_transform(X_train)

### We have 50 input features and one target variable. 
### 2 Hidden layers. Each hidden layer will have 25 nodes.

ReLu will be the activation function for hidden layers. As this is a binary classification problem we will use sigmoid as the activation function.

In [9]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(25, activation='relu', kernel_initializer='random_normal', input_dim=50))
#Second  Hidden Layer
classifier.add(Dense(25, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [10]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [11]:
#Fitting the data to the training dataset
classifier.fit(X_train,y_train, batch_size=10, epochs=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
104051/104051 [==============================] - 10s 100us/step - loss: 0.3414 - acc: 0.8521
Epoch 2/100
104051/104051 [==============================] - 10s 97us/step - loss: 0.2992 - acc: 0.8735
Epoch 3/100
104051/104051 [==============================] - 10s 95us/step - loss: 0.2772 - acc: 0.8840
Epoch 4/100
104051/104051 [==============================] - 10s 96us/step - loss: 0.2585 - acc: 0.8924
Epoch 5/100
104051/104051 [==============================] - 10s 96us/step - loss: 0.2465 - acc: 0.8982
Epoch 6/100
104051/104051 [==============================] - 10s 97us/step - loss: 0.2380 - acc: 0.9025
Epoch 7/100
104051/104051 [==============================] - 10s 95us/step - loss: 0.2326 - acc: 0.9041
Epoch 8/100
104051/104051 [==============================] - 10s 97us/step - loss: 0.2279 - acc: 0.9067
Epoch 9/100
104051/104051 [==============================] - 10s 95us/step - loss: 0.2260 - acc: 0.9070
Epoch 10/100
10

104051/104051 [==============================] - 10s 97us/step - loss: 0.1694 - acc: 0.9311
Epoch 76/100
104051/104051 [==============================] - 10s 99us/step - loss: 0.1699 - acc: 0.9307 1s - loss: 0.1706 - acc: 0.9 - ETA: 1s - loss
Epoch 77/100
104051/104051 [==============================] - 11s 104us/step - loss: 0.1689 - acc: 0.9310
Epoch 78/100
104051/104051 [==============================] - 10s 97us/step - loss: 0.1694 - acc: 0.9309
Epoch 79/100
104051/104051 [==============================] - 10s 98us/step - loss: 0.1687 - acc: 0.9310
Epoch 80/100
104051/104051 [==============================] - 10s 96us/step - loss: 0.1693 - acc: 0.9309
Epoch 81/100
104051/104051 [==============================] - 10s 97us/step - loss: 0.1688 - acc: 0.9315
Epoch 82/100
104051/104051 [==============================] - 10s 97us/step - loss: 0.1685 - acc: 0.9312
Epoch 83/100
104051/104051 [==============================] - 10s 97us/step - loss: 0.1677 - acc: 0.9322
Epoch 84/100
104051/1

In [12]:
eval_model=classifier.evaluate(X_train, y_train)
eval_model

104051/104051 [==============================] - 1s 11us/step


[0.16577561138220748, 0.9322255432472272]

In [13]:
y_pred=classifier.predict(X_test)
y_pred =(y_pred>0.5)

In [14]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[18658     0]
 [ 7355     0]]


In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.717256756237266

## Tune NN

In [16]:
classifier.fit(X_train,y_train, batch_size=100, epochs=100)

Epoch 1/100
104051/104051 [==============================] - 1s 11us/step - loss: 0.1573 - acc: 0.9369
Epoch 2/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1573 - acc: 0.9365
Epoch 3/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1574 - acc: 0.9368
Epoch 4/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1574 - acc: 0.9361
Epoch 5/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1576 - acc: 0.9361
Epoch 6/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1579 - acc: 0.9362
Epoch 7/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1575 - acc: 0.9360
Epoch 8/100
104051/104051 [==============================] - 1s 11us/step - loss: 0.1576 - acc: 0.9363
Epoch 9/100
104051/104051 [==============================] - 1s 11us/step - loss: 0.1570 - acc: 0.9371
Epoch 10/100
104051/104051 [==============================] - 1s 10us/ste

104051/104051 [==============================] - 1s 10us/step - loss: 0.1540 - acc: 0.9380
Epoch 79/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1537 - acc: 0.9376
Epoch 80/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1533 - acc: 0.9383
Epoch 81/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1536 - acc: 0.9380
Epoch 82/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1538 - acc: 0.9378
Epoch 83/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1535 - acc: 0.9381
Epoch 84/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1534 - acc: 0.9383
Epoch 85/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1538 - acc: 0.9383
Epoch 86/100
104051/104051 [==============================] - 1s 10us/step - loss: 0.1537 - acc: 0.9386
Epoch 87/100
104051/104051 [==============================] - 1s 10us/step - 